In [1]:
from os import environ as ENV
ENV['CUDA_VISIBLE_DEVICES'] = '0'

import pennylane as qml
import jax.numpy as jnp
import jax
import numpy as np
import numpy.typing as npt

print(jax.devices())
# probably wont matter
# jax.config.update("jax_enable_x64", True)

# NOTE: if you remove this, re-run the notebook SEVERAL times because it might break after a few
from catalyst.utils.runtime_environment import get_cli_path

dev = qml.device("lightning.gpu", wires=18)

[cuda(id=0)]


In [2]:
import diskcache as dc
cache = dc.Cache(
    ENV['PWD'] + '/.cache_pennylane_sim/',
    cull_limit = 0,
    size_limit = 2 ** 32, # 4GB
    timeout = 60**2 * 24 * 7, # 1 week
)

In [10]:
import sys
sys.path.append('..')

from circuit_postprocess import *
from should_be_stdlib import *

In [3]:
# if either is the 0 vector, then norm is undefined. unlikely to occur
@qml.qjit
@qml.qnode(dev)
def swap_circuit(data1:npt.NDArray[np.float64], data2:npt.NDArray[np.float64], norm:bool=True) -> npt.NDArray[np.float64]:
    l1, l2 = len(data1), len(data2)
    assert l1 == l2

    # normalize input data
    data1 = data1 / jnp.linalg.norm(data1, ord=1) * jnp.pi/2
    data2 = data2 / jnp.linalg.norm(data2, ord=1) * jnp.pi/2
    # print(data1)
    # print(data2)

    # data embedding
    for i in range(l1 + l2):
        qml.H(i)
    for i in range(l1):
        qml.RY(data1[i], i)
    for i in range(l2):
        qml.RY(data2[i], l1 + i)

    # QFT is inv DFT; inv QFT is DFT
    qml.QFT(wires = range(l1))
    qml.QFT(wires = range(l1, l1 + l2))

    # Swap test
    for i in range(l1):
        qml.CNOT(wires = [i, l1 + i])
    qml.Barrier()
    for i in range(l1):
        qml.H(i)

    return qml.probs(wires = range(l1 + l2))

In [6]:
#TODO: memoize over the neuron idx

@cache.memoize()
def get_fidelity_memo(data1:list[int], data2:list[int]) -> tuple[float,float]:
    probs = swap_circuit(data1, data2)
    return swap_expectation(probs, len(data1))

def get_fidelity(data1:list[int], data2:list[int]) -> tuple[float,float]:
    # data1 should be less than data2
    if is_array_lesser(data2, data1):
        return get_fidelity(data2, data1)
    # normalize data
    data1 = data1 / np.linalg.norm(data1, ord=1) * np.pi/2
    data2 = data2 / np.linalg.norm(data2, ord=1) * np.pi/2
    return get_fidelity_memo(data1, data2)

In [7]:
a = jnp.array([2,2,2, 2,2,2, 2,2,2,])
b = jnp.array([2,2,1, 2,2,2, 2,10,1,])

In [8]:
print('Drawing')
print(qml.draw(swap_circuit)(a, b))

Drawing
 0: ──H──RY(0.17)─╭QFT─╭●──────────────────────────||──H─┤ ╭Probs
 1: ──H──RY(0.17)─├QFT─│──╭●───────────────────────||──H─┤ ├Probs
 2: ──H──RY(0.17)─├QFT─│──│──╭●────────────────────||──H─┤ ├Probs
 3: ──H──RY(0.17)─├QFT─│──│──│──╭●─────────────────||──H─┤ ├Probs
 4: ──H──RY(0.17)─├QFT─│──│──│──│──╭●──────────────||──H─┤ ├Probs
 5: ──H──RY(0.17)─├QFT─│──│──│──│──│──╭●───────────||──H─┤ ├Probs
 6: ──H──RY(0.17)─├QFT─│──│──│──│──│──│──╭●────────||──H─┤ ├Probs
 7: ──H──RY(0.17)─├QFT─│──│──│──│──│──│──│──╭●─────||──H─┤ ├Probs
 8: ──H──RY(0.17)─╰QFT─│──│──│──│──│──│──│──│──╭●──||──H─┤ ├Probs
 9: ──H──RY(0.13)─╭QFT─╰X─│──│──│──│──│──│──│──│───||────┤ ├Probs
10: ──H──RY(0.13)─├QFT────╰X─│──│──│──│──│──│──│───||────┤ ├Probs
11: ──H──RY(0.07)─├QFT───────╰X─│──│──│──│──│──│───||────┤ ├Probs
12: ──H──RY(0.13)─├QFT──────────╰X─│──│──│──│──│───||────┤ ├Probs
13: ──H──RY(0.13)─├QFT─────────────╰X─│──│──│──│───||────┤ ├Probs
14: ──H──RY(0.13)─├QFT────────────────╰X─│──│──│───||────┤ ├Probs
15

In [9]:
exp_swap, fidelity = get_fidelity(a, b)
exp_swap, fidelity

(Array(1., dtype=float64), Array(1., dtype=float64))

# TODO

- memoize functions
- run on GPU
- run on QPU (no mitigation)
- run on QPU (with Mitiq)
- end to end analysis